In [1]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import datetime
import numpy as np
from fastapi import FastAPI, Query
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# Load your dataset (assuming it's in a CSV file)
data = pd.read_csv("unique_dates_cities_df.csv")

# data = data[:10000]
data = data.drop("Unnamed: 0", axis=1)

# Preprocessing
# Assuming your dataset has columns: 'datetime', 'city', 'count'
# Convert the 'datetime' column to a proper datetime format
data['datetime'] = pd.to_datetime(data['datetime'])

# Extract relevant features from the datetime column
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['day_of_week'] = data['datetime'].dt.dayofweek  # Monday = 0, Sunday = 6

# One-hot encode the 'city' column
data = pd.get_dummies(data, columns=['city'], prefix='city')

# Split the data
X = data.drop(["count", "datetime"], axis=1)  # Features
y = data["count"]  # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
data.shape

(66050, 3584)

In [11]:
# def polynomial_model_training(degree):
#     # Generate polynomial features
#     poly = PolynomialFeatures(degree=degree)
#     X_train_poly = poly.fit_transform(X_train)
#     X_test_poly = poly.transform(X_test)
# 
#     # Model Training
#     model = LinearRegression()
#     model.fit(X_train_poly, y_train)
# 
#     # Model Evaluation
#     y_pred = model.predict(X_test_poly)
#     mse = mean_squared_error(y_test, y_pred)
#     rmse = np.sqrt(mse)
#     print(f"Root Mean Squared Error (Polynomial Degree {degree}): {rmse}")


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from tqdm import tqdm

# Initialize the Gradient Boosting Regressor
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model with a progress bar
n_iterations = 100  # Number of training iterations
with tqdm(total=n_iterations, desc="Training") as pbar:
    for _ in range(n_iterations):
        model.fit(X_train, y_train)
        # Update the progress bar
        pbar.update(1)

joblib.dump(model, 'models/randomForest_regression_model.pkl')

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Training: 100%|██████████| 100/100 [6:49:41<00:00, 245.81s/it] 


Mean Squared Error: 982.8303136657373


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np

def model_training(X_train, y_train, X_test, y_test):
    # Create a RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators=1000, random_state=42)

    # Model Training with Progress Bar
    epochs = 10  # Set the number of epochs or iterations
    with tqdm(total=epochs, desc="Training Progress") as pbar:
        for epoch in range(epochs):
            regressor.fit(X_train, y_train)
            pbar.update(1)  # Update the progress bar

    joblib.dump(regressor, 'models/randomForest_regression_model.pkl')
    
    # Model Evaluation
    y_pred = regressor.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error with RandomForestRegressor: {rmse}")

# Replace X_train, y_train, X_test, y_test with your actual data
model_training(X_train, y_train, X_test, y_test)

Training Progress: 100%|██████████| 10/10 [00:25<00:00,  2.51s/it]


Root Mean Squared Error with RandomForestRegressor: 0.8660254037844386


In [9]:
def save_model():
    # Save the model to a file
    joblib.dump(regressor, 'randomForest_regression_model.pkl')

save_model()

NameError: name 'model' is not defined

In [ ]:
# Load the saved model from the file
model = joblib.load('models/polynomial_regression_model.pkl')